In [1]:
import pandas as pd
import os

In [2]:
csv_file = os.path.expanduser("~/Desktop/articles1.csv")
articles1 = pd.read_csv(csv_file, encoding="utf-8")

In [3]:
csv_file = os.path.expanduser("~/Desktop/articles2.csv")
articles2 = pd.read_csv(csv_file, encoding="utf-8")

In [4]:
csv_file = os.path.expanduser("~/Desktop/articles3.csv")
articles3 = pd.read_csv(csv_file, encoding="utf-8")

In [5]:
merged_articles = pd.concat([articles1, articles2, articles3])

In [6]:
merged_articles.drop(columns=["Unnamed: 0", "id", "year", "month"], inplace=True)

In [7]:
merged_articles['date'] = pd.to_datetime(merged_articles['date'])
merged_articles = merged_articles.sort_values(by='date', ascending=True)

In [18]:
unique_publications = merged_articles['publication'].unique()
print(unique_publications)

['NPR' 'Talking Points Memo' 'Fox News' 'National Review' 'Guardian'
 'Washington Post' 'CNN' 'Reuters' 'Breitbart' 'New York Post' 'Vox'
 'New York Times' 'Business Insider' 'Buzzfeed News' 'Atlantic']


In [19]:
publication_counts = merged_articles['publication'].value_counts()
print(publication_counts)

Breitbart              23781
New York Post          17493
NPR                    11992
CNN                    11488
Washington Post        11114
Reuters                10710
Guardian                8681
New York Times          7803
Atlantic                7179
Business Insider        6757
National Review         6203
Talking Points Memo     5214
Vox                     4947
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64


In [4]:
csv_file = os.path.expanduser("~/Desktop/abcnews-date-text.csv")
abcnews = pd.read_csv(csv_file, encoding="utf-8")

In [8]:
json_string = merged_articles.to_json(orient="records", date_format="iso")

In [9]:
import json
json_data = json.loads(json_string)

In [11]:
import pandas as pd

# Convert json_data back to a DataFrame
df = pd.DataFrame(json_data)

# Convert the date column back to datetime format
df['date'] = pd.to_datetime(df['date'])

# Save the DataFrame to a CSV file
df.to_csv(os.path.expanduser("~/Desktop/articles.csv"), index=False)

In [11]:
import paho.mqtt.client as mqtt
import json

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    test_message = {"key": "value"}  # Simple JSON message
    client.publish("your/topic", json.dumps(test_message))
    print("Sent test message:", test_message)

client = mqtt.Client()
client.on_connect = on_connect

client.username_pw_set("username", "password")  # If required
client.connect("127.0.0.1", 1883, 60)

client.loop_forever()

Connected with result code 0
Sent test message: {'key': 'value'}


KeyboardInterrupt: 

In [9]:
import paho.mqtt.client as mqtt
import json
import time

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    for record in json_data:  # Iterate over the records in json_data
        client.publish("your/topic", json.dumps(record))
        print(f"Sent record: {record}")  # Print the sent record
        time.sleep(1)  # Wait for 1 second

client = mqtt.Client()
client.on_connect = on_connect

client.username_pw_set("username", "password")  # If required
client.connect("127.0.0.1", 1883, 60)

client.loop_forever()

Connected with result code 0
Sent record: {'publish_date': 20030219, 'headline_text': 'aba decides against community broadcasting licence'}
Sent record: {'publish_date': 20030219, 'headline_text': 'act fire witnesses must be aware of defamation'}
Sent record: {'publish_date': 20030219, 'headline_text': 'a g calls for infrastructure protection summit'}
Sent record: {'publish_date': 20030219, 'headline_text': 'air nz staff in aust strike for pay rise'}
Sent record: {'publish_date': 20030219, 'headline_text': 'air nz strike to affect australian travellers'}
Sent record: {'publish_date': 20030219, 'headline_text': 'ambitious olsson wins triple jump'}
Sent record: {'publish_date': 20030219, 'headline_text': 'antic delighted with record breaking barca'}
Sent record: {'publish_date': 20030219, 'headline_text': 'aussie qualifier stosur wastes four memphis match'}
Sent record: {'publish_date': 20030219, 'headline_text': 'aust addresses un security council over iraq'}
Sent record: {'publish_date

KeyboardInterrupt: 